### Evaluation Results -> Succ Rate

In [2]:
import os
import os.path as osp
import numpy as np
import pandas as pd

In [4]:
root = '/Users/jimmy/Projects/RL/My_RLlib_Algo/eval_results'
all_file_names = os.listdir(root)

In [3]:
test_csv = pd.read_csv(osp.join(root, 'scpo_inter_0.csv'))
test_csv.head()

NameError: name 'root' is not defined

In [5]:
test_csv.columns

Index(['Unnamed: 0', 'velocity_step_mean_episode_min',
       'velocity_step_mean_episode_mean', 'velocity_step_mean_episode_max',
       'energy_step_mean_episode_min', 'energy_step_mean_episode_mean',
       'energy_step_mean_episode_max', 'num_neighbours_mean_episode_mean',
       'num_neighbours_mean_episode_max', 'num_agents_total',
       'num_agents_total_per_300_steps', 'success_rate', 'num_agents_success',
       'num_agents_success_per_300_steps', 'num_agents_failed_per_300_steps',
       'episode_reward_mean', 'episode_reward_min', 'episode_reward_max',
       'episode_cost_mean', 'episode_cost_min', 'episode_cost_max',
       'episode_cost_sum', 'crash_rate', 'num_agents_crash', 'out_rate',
       'num_agents_out', 'episode_length_mean', 'success_episode_length_mean',
       'svo_estimate_deg_mean', 'svo_estimate_deg_min', 'svo_estimate_deg_max',
       'svo_reward', 'path', 'ckpt_idx', 'algo', 'env', 'seed', 'trial',
       'trial_path', 'should_wrap_copo_env', 'should_wra

`concat` 一个 algo scene 对应的所有 eval csv 文件并保留有用的列，然后返回

In [24]:
def one_algo_scene(algo, scene, root=root):
    algo_scene = f'{algo}_{scene}'
    count = 0
    succ = [] # 对所有一次性求std
    crash = []
    out = []
    all_file_names = os.listdir(root)
    csv_list = []
    for name in all_file_names:
        if algo_scene in name:
            count += 1
            csv_list.append(pd.read_csv(osp.join(root, name)))
    print(f'Found {count} eval results(csv) of {algo_scene}.')

    data = pd.concat(csv_list)

    ret = pd.DataFrame(columns=['success_rate', 'num_failed', 'net_success'])

    # == Success Rate ==
    ret['success_rate'] = data['success_rate']*100
    # == Safety ==
    # ret["num_failed"] = (data["num_agents_crash"] + data["num_agents_out"] ) / data['num_agents_total']
    ret["num_failed"] = (data["crash_rate"] + data["out_rate"])*100
    # ret["num_failed"] = (data["num_agents_total"] - data["num_agents_success"]) / data['num_agents_total'] * 100
    # == Efficiency ==
    data['num_agents_maxstep'] = data["num_agents_total"] - data["num_agents_success"] - data["num_agents_crash"] - data["num_agents_out"]
    ret["net_success"] = (
        # data["num_agents_success"] - data["num_agents_crash"] - data["num_agents_out"] - data['num_agents_maxstep']
        data["num_agents_success"] -ret["num_failed"]
        # data["num_agents_success"] - data["num_agents_crash"] - data["num_agents_out"]
    )

    return ret


In [61]:
ret = one_algo_scene('ippo', 'inter')
ret.columns

Found 24 eval results(csv) of ippo_inter.


Index(['success_rate', 'num_failed', 'net_success'], dtype='object')

In [25]:
algos = [ 'ippo', 'ccppo-mf', 'ccppo-concat', 'copo', 'scpo']
algo_names_formal = ['IPPO', 'MFPO', 'MAPPO', 'CoPO', 'SCPO']
scenes = ['inter', 'round']
scene_names_formal = ['Intersection', 'Roundabout']
metric_names = ['Success Rate', 'Safety Violation', 'Efficiency']

algo_mapping = {
    'ippo': 'IPPO', 
    'ccppo-mf': 'MFPO',
    'ccppo-concat': 'MAPPO', 
    'copo': 'CoPO', 
    'scpo': 'SCPO', 
}

scene_mapping = {
    'inter': 'Intersection',
    'round': 'Roundabout',
}

metric_mapping = {
    'success_rate': 'Success Rate',
    'num_failed': 'Safety Violation', 
    'net_success': 'Efficiency',
}

# Method(5)  Metrics(3) -> 5x3 
table_results_inter = pd.DataFrame(columns=metric_names)
table_results_round = pd.DataFrame(columns=metric_names)

for algo in algos:
    for scene in scenes:
        df = one_algo_scene(algo, scene) # 'success_rate', 'num_failed', 'net_success'
        # print(f'{algo}_{scene}', rate, sum(rate))
        table_results = table_results_inter if scene == 'inter' else table_results_round
        for met in ['success_rate', 'num_failed', 'net_success']:
            table_results.loc[algo_mapping[algo], metric_mapping[met]] = f"{df[met].mean():.2f} $\scriptstyle \pm {df[met].std():.2f}$"


Found 24 eval results(csv) of ippo_inter.
Found 24 eval results(csv) of ippo_round.
Found 40 eval results(csv) of ccppo-mf_inter.
Found 40 eval results(csv) of ccppo-mf_round.
Found 40 eval results(csv) of ccppo-concat_inter.
Found 40 eval results(csv) of ccppo-concat_round.
Found 34 eval results(csv) of copo_inter.
Found 24 eval results(csv) of copo_round.
Found 40 eval results(csv) of scpo_inter.
Found 36 eval results(csv) of scpo_round.


In [26]:
print(table_results_inter)
# print(table_results_round)

                         Success Rate                Safety Violation  \
IPPO   73.33 $\scriptstyle \pm 14.01$  24.07 $\scriptstyle \pm 10.40$   
MFPO   70.09 $\scriptstyle \pm 16.64$  27.58 $\scriptstyle \pm 14.00$   
MAPPO  69.93 $\scriptstyle \pm 12.55$   28.39 $\scriptstyle \pm 9.93$   
CoPO    76.31 $\scriptstyle \pm 7.44$   23.33 $\scriptstyle \pm 6.57$   
SCPO    82.28 $\scriptstyle \pm 7.51$   16.56 $\scriptstyle \pm 6.40$   

                           Efficiency  
IPPO   50.75 $\scriptstyle \pm 30.98$  
MFPO   46.45 $\scriptstyle \pm 36.89$  
MAPPO  47.43 $\scriptstyle \pm 30.65$  
CoPO   61.51 $\scriptstyle \pm 16.45$  
SCPO   72.76 $\scriptstyle \pm 18.24$  


In [27]:
print(table_results_round)

                         Success Rate               Safety Violation  \
IPPO   75.92 $\scriptstyle \pm 12.70$  13.20 $\scriptstyle \pm 6.63$   
MFPO    79.50 $\scriptstyle \pm 9.45$  13.31 $\scriptstyle \pm 5.71$   
MAPPO  71.81 $\scriptstyle \pm 11.98$  12.74 $\scriptstyle \pm 6.43$   
CoPO    78.67 $\scriptstyle \pm 8.68$  17.05 $\scriptstyle \pm 6.77$   
SCPO    82.96 $\scriptstyle \pm 6.71$  15.53 $\scriptstyle \pm 6.43$   

                           Efficiency  
IPPO   54.49 $\scriptstyle \pm 23.32$  
MFPO   59.54 $\scriptstyle \pm 17.26$  
MAPPO  45.43 $\scriptstyle \pm 18.89$  
CoPO   56.88 $\scriptstyle \pm 17.06$  
SCPO   69.95 $\scriptstyle \pm 14.10$  


In [58]:
print(table_results_inter.to_latex(escape=False))

\begin{tabular}{llll}
\toprule
 & Success Rate & Safety Violation & Efficiency \\
\midrule
IPPO & 73.33 $\scriptstyle \pm 14.01$ & 24.97 $\scriptstyle \pm 10.24$ & 51.51 $\scriptstyle \pm 27.31$ \\
MFPO & 70.09 $\scriptstyle \pm 16.64$ & 29.11 $\scriptstyle \pm 13.27$ & 46.61 $\scriptstyle \pm 32.63$ \\
MAPPO & 69.93 $\scriptstyle \pm 12.55$ & 30.34 $\scriptstyle \pm 9.03$ & 46.67 $\scriptstyle \pm 26.35$ \\
CoPO & 76.31 $\scriptstyle \pm 7.44$ & 26.23 $\scriptstyle \pm 7.97$ & 58.88 $\scriptstyle \pm 15.36$ \\
SCPO & 82.28 $\scriptstyle \pm 7.51$ & 19.02 $\scriptstyle \pm 8.00$ & 71.27 $\scriptstyle \pm 17.07$ \\
\bottomrule
\end{tabular}



In [59]:
print(table_results_round.to_latex(escape=False))

\begin{tabular}{llll}
\toprule
 & Success Rate & Safety Violation & Efficiency \\
\midrule
IPPO & 75.92 $\scriptstyle \pm 12.70$ & 19.79 $\scriptstyle \pm 8.96$ & 56.24 $\scriptstyle \pm 21.69$ \\
MFPO & 79.50 $\scriptstyle \pm 9.45$ & 17.82 $\scriptstyle \pm 6.50$ & 60.59 $\scriptstyle \pm 16.00$ \\
MAPPO & 71.81 $\scriptstyle \pm 11.98$ & 21.58 $\scriptstyle \pm 7.74$ & 47.98 $\scriptstyle \pm 17.28$ \\
CoPO & 78.67 $\scriptstyle \pm 8.68$ & 19.55 $\scriptstyle \pm 7.33$ & 57.89 $\scriptstyle \pm 15.88$ \\
SCPO & 82.96 $\scriptstyle \pm 6.71$ & 17.59 $\scriptstyle \pm 7.26$ & 69.17 $\scriptstyle \pm 13.61$ \\
\bottomrule
\end{tabular}



In [50]:
def aggregate_results(root):
    files = os.listdir(root)

    all_mean = []
    all_res = []
    for f in files:
        fp = osp.join(root, f)
        data = pd.read_csv(fp)
        seed = data['seed'].unique()
        ckpt_idx = data['ckpt_idx'].unique()
    
        mean = data['success_rate'].mean()
        all_mean.append(mean)
        res = f'{mean:.3} - {seed}, {ckpt_idx}'
        all_res.append(res)

    all_mean.sort()
    all_res.sort(key=lambda x: float(x.split('-')[0][:-1]))

    return all_mean, all_res